# Imports

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Lambda, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


ModuleNotFoundError: No module named 'tensorflow'

# Preprocessing Data
- Takes the CIFAR-10 dataset (X, Y) and preprocesses it, resizes the images and converts the labels to one-hot vectors.
- The first line of code creates a new array X_p with the same shape and content as X, but with the data type converted to float32.
- Next, we normalize image data by dividing each pixel value by 255. This scales the pixel values to the range [0, 1].
- Next, we resize images to 224x224 pixels (the required size for ResNet50 model). Convert back to numpy array.
- Convert labels into one-hot vectors using keras 'to_categorical' function. In this case, each label is represented as a binary vector of length 10 (the number of classes in CIFAR-10), with a 1 at the index corresponding to the class and 0s elsewhere.
- Finally, the function returns the preprocessed image data (X_p) and labels (Y_p) as a tuple.

**In summary, the preprocess_data function converts the image data to float32, normalizes it, resizes the images, and converts the labels to one-hot encoding. These preprocessing steps are crucial for preparing the dataset for training a CNN model, such as the one used in this task.**

In [3]:
def preprocess_data(X, Y):
    """preprocesses the data for your model"""
    X_p = X.astype('float32')
    X_p /= 255
    X_p = tf.image.resize(X_p, (224, 224)).numpy()

    Y_p = to_categorical(Y, 10)

    return X_p, Y_p

# Create Model